In [12]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter
import mymodel

In [13]:
set_frame = 50

epochs = 60
batch_sz = 10
checkpoint_frequency = 3
learning_rate = 0.00005
gamma = 0.5


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dl, val_dl, test_dl = mymodel.load_data()

dataloaders = {
    'train' : train_dl,
    'val' : val_dl,
    'test' : test_dl
}



network = mymodel.get_simple_conv_net()
network = network.to(DEVICE)

optim = SGD(network.parameters(), lr=learning_rate)
lr_sch = ExponentialLR(optim, gamma)

writer = SummaryWriter()
stopp = mymodel.EarlyStopper(patience = 5, tolerance = 2)

In [33]:

mymodel.train_model(network,epochs,dataloaders,optim,lr_sch,writer,stopp,checkpoint_frequency)

Train epoch[1/60], MiniBatch[20/24], Loss: 2.57022, Acc: 9.00000, LR: 0.00005
Eval epoch[1/60], MiniBatch[4/4], Loss: 3.14961, Acc: 6.25000                
Train epoch[2/60], MiniBatch[20/24], Loss: 2.43751, Acc: 11.00000, LR: 0.00003
Eval epoch[2/60], MiniBatch[4/4], Loss: 2.40664, Acc: 0.00000                 
Train epoch[3/60], MiniBatch[20/24], Loss: 2.29651, Acc: 7.00000, LR: 0.00001
Eval epoch[3/60], MiniBatch[4/4], Loss: 2.14750, Acc: 12.50000, Epochs without improvement: 1
Train epoch[4/60], MiniBatch[20/24], Loss: 2.13974, Acc: 19.00000, LR: 0.00001               
Eval epoch[4/60], MiniBatch[4/4], Loss: 2.79832, Acc: 37.50000                
Train epoch[5/60], MiniBatch[20/24], Loss: 2.26063, Acc: 29.00000, LR: 0.00000
Eval epoch[5/60], MiniBatch[4/4], Loss: 2.04348, Acc: 37.50000                
Train epoch[6/60], MiniBatch[20/24], Loss: 2.13804, Acc: 22.00000, LR: 0.00000
Eval epoch[6/60], MiniBatch[4/4], Loss: 2.40010, Acc: 37.50000                
Train epoch[7/60], MiniBa

In [22]:
def train_model_gpu_lr_conv_valid(model, epochs, dataloaders, optimiser, lr_scheduler):
    msg = ''
    for epoch in range(epochs):
        ################ TRAINING ################
        model.train()
        train_dl = dataloaders["train"]
        correct_train = 0
        total_train = 0

        for batch_num, (motion_batch, label_batch) in enumerate(train_dl):
            batch_sz = len(motion_batch)
            label_batch = label_batch.to(DEVICE)
            motion_batch = motion_batch.to(DEVICE)
            output = model(motion_batch)
            losses = nn.CrossEntropyLoss()(output, label_batch)

            optimiser.zero_grad()
            losses.backward()
            optimiser.step()

            preds_train = torch.argmax(output, dim=1)
            correct_train += int(torch.eq(preds_train, label_batch).sum())
            total_train +=batch_sz
            minibatch_accuracy_train = 100 * correct_train / total_train

            #### Fancy printing stuff, you can ignore this! ######
            if (batch_num + 1) % 5 == 0:
                print(" " * len(msg), end='\r')
                msg = f'Train epoch[{epoch+1}/{epochs}], MiniBatch[{batch_num + 1}/{total_train}], Loss: {losses.item():.5f}, Acc: {minibatch_accuracy_train:.5f}, LR: {lr_scheduler.get_last_lr()[0]:.5f}'
                print (msg, end='\r' if epoch < epochs else "\n",flush=True)
            #### Fancy printing stuff, you can ignore this! ######
        lr_scheduler.step()

        ##############################################################
        print("") # Create newline between progress bars
        ##############################################################

        model.eval()
        val_dl = dataloaders["val"]
        total_val = len(val_dl)
        correct_val = 0
        total_val = 0

        for batch_num, (motion_batch, label_batch) in enumerate(val_dl):
            batch_sz = len(motion_batch)
            label_batch = label_batch.to(DEVICE)
            motion_batch = motion_batch.to(DEVICE)
            with torch.no_grad():
                output = model(motion_batch)
                losses = nn.CrossEntropyLoss()(output, label_batch)
                preds_val = torch.argmax(output, dim=1)
                correct_val += int(torch.eq(preds_val, label_batch).sum())
                total_val += batch_sz
                minibatch_accuracy_val = 100 * correct_val / total_val
            #### Fancy printing stuff, you can ignore this! ######
            #if (batch_num + 1) % 5 == 0:
            print(" " * len(msg), end='\r')
            msg = f'Val epoch[{epoch+1}/{epochs}], MiniBatch[{batch_num + 1}/{total_val}], Loss: {losses.item():.5f}, Acc: {minibatch_accuracy_val:.5f}, LR: {lr_scheduler.get_last_lr()[0]:.5f}'
            print (msg, end='\r' if epoch < epochs else "\n",flush=True)
            #### Fancy printing stuff, you can ignore this! ######
     
        ########################################################################
        print("")  # Create newline between progress bars

In [23]:
# instantiate simple conv net
network = get_simple_conv_net()
# instantiate SGD optimiser
optim = SGD(network.parameters(),lr = learning_rate)
# instantiate exponential learning rate scheduler
lr_sch = ExponentialLR(optim,gamma)
# move model to DEVICE
network = network.to(DEVICE)
# call latest training function
train_model_gpu_lr_conv_valid(network,epochs,dataloaders,optim,lr_sch)

Train epoch[1/60], MiniBatch[20/100], Loss: 2.88297, Acc: 17.00000, LR: 0.00005
Val epoch[1/60], MiniBatch[4/16], Loss: 1.82018, Acc: 6.25000, LR: 0.00003     
Train epoch[2/60], MiniBatch[20/100], Loss: 2.47850, Acc: 11.00000, LR: 0.00003
Val epoch[2/60], MiniBatch[4/16], Loss: 2.28179, Acc: 12.50000, LR: 0.00001    
Train epoch[3/60], MiniBatch[20/100], Loss: 2.26144, Acc: 20.00000, LR: 0.00001
Val epoch[3/60], MiniBatch[4/16], Loss: 2.10723, Acc: 12.50000, LR: 0.00001    
Train epoch[4/60], MiniBatch[20/100], Loss: 2.37366, Acc: 22.00000, LR: 0.00001
Val epoch[4/60], MiniBatch[4/16], Loss: 2.42516, Acc: 12.50000, LR: 0.00000    
Train epoch[5/60], MiniBatch[20/100], Loss: 2.19370, Acc: 22.00000, LR: 0.00000
Val epoch[5/60], MiniBatch[4/16], Loss: 2.24689, Acc: 12.50000, LR: 0.00000    
Train epoch[6/60], MiniBatch[20/100], Loss: 2.08620, Acc: 20.00000, LR: 0.00000
Val epoch[6/60], MiniBatch[4/16], Loss: 2.49619, Acc: 12.50000, LR: 0.00000    
Train epoch[7/60], MiniBatch[20/100], Lo

KeyboardInterrupt: 